In [13]:

import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [20]:
data=pd.read_csv('./Building-Week-2/data/data_5secondWindow .csv')

In [22]:
data.shape

(5893, 70)

In [23]:
def change_name(df):
    column_names=[]
    for i in df.columns:
        k=i.replace('.','_').replace("#",'_')
        df.rename(columns = {i:k}, inplace = True)
    return df
data=change_name(data)

In [24]:
# create test data from some user 
df=data.copy()
user1=df[df['user']=='Pierpaolo']
user2=df[df['user']=='IvanHeibi']
user3=df[df['user']=='AndreaCarpineti']
user4=df[df['user']=='Elena']
users=[user1,user2,user3,user4]
df_test=pd.concat(users)
df_test.head(5)

,id,time,activityrecognition_0,activityrecognition_1,android_sensor_accelerometer_mean,android_sensor_accelerometer_min,android_sensor_accelerometer_max,android_sensor_accelerometer_std,android_sensor_game_rotation_vector_mean,android_sensor_game_rotation_vector_min,android_sensor_game_rotation_vector_max,android_sensor_game_rotation_vector_std,android_sensor_gravity_mean,android_sensor_gravity_min,android_sensor_gravity_max,android_sensor_gravity_std,android_sensor_gyroscope_mean,android_sensor_gyroscope_min,android_sensor_gyroscope_max,android_sensor_gyroscope_std,android_sensor_gyroscope_uncalibrated_mean,android_sensor_gyroscope_uncalibrated_min,android_sensor_gyroscope_uncalibrated_max,android_sensor_gyroscope_uncalibrated_std,android_sensor_light_mean,android_sensor_light_min,android_sensor_light_max,android_sensor_light_std,android_sensor_linear_acceleration_mean,android_sensor_linear_acceleration_min,android_sensor_linear_acceleration_max,android_sensor_linear_acceleration_std,android_sensor_magnetic_field_mean,android_sensor_magnetic_field_min,android_sensor_magnetic_field_max,android_sensor_magnetic_field_std,android_sensor_magnetic_field_uncalibrated_mean,android_sensor_magnetic_field_uncalibrated_min,android_sensor_magnetic_field_uncalibrated_max,android_sensor_magnetic_field_uncalibrated_std,android_sensor_orientation_mean,android_sensor_orientation_min,android_sensor_orientation_max,android_sensor_orientation_std,android_sensor_pressure_mean,android_sensor_pressure_min,android_sensor_pressure_max,android_sensor_pressure_std,android_sensor_proximity_mean,android_sensor_proximity_min,android_sensor_proximity_max,android_sensor_proximity_std,android_sensor_rotation_vector_mean,android_sensor_rotation_vector_min,android_sensor_rotation_vector_max,android_sensor_rotation_vector_std,android_sensor_step_counter_mean,android_sensor_step_counter_min,android_sensor_step_counter_max,android_sensor_step_counter_std,sound_mean,sound_min,sound_max,sound_std,speed_mean,speed_min,speed_max,speed_std,target,user
33,19919,260.0,NaN,5.0,11.068949,9.764513,14.028126,1.283204,0.943836,0.936799,0.956021,0.010595,9.806651,9.806651,9.806652,2.297380e-07,0.615399,0.051965,2.028358,0.655962,0.778458,0.320857,1.253073,0.441447,NaN,NaN,NaN,NaN,1.721314,1.721314,1.721314,0.852304,39.061158,34.090004,44.518024,3.797039,53.455853,36.378866,66.912548,13.382351,258.020318,166.195530,308.596876,62.916080,1011.91840,1011.91840,1011.91840,0.006604,NaN,NaN,NaN,NaN,0.955202,0.955202,0.955202,0.030968,29514.0,29514.0,29514.0,NaN,77.064227,77.064227,77.064227,4.549349,1.191050,1.191050,1.191050,NaN,Walking,Pierpaolo
54,19494,78.0,NaN,75.0,9.263056,8.712177,9.963264,0.361961,0.894102,0.894102,0.894102,0.002254,9.806650,9.806650,9.806650,9.326844e-07,0.447802,0.318240,0.544166,0.116564,0.334574,0.132206,0.742806,0.222843,NaN,NaN,NaN,NaN,0.958200,0.696447,1.200224,0.252467,75.575010,73.195268,79.696264,1.973862,77.938475,77.496043,78.380906,0.625692,256.163942,233.331217,276.149317,17.573159,1011.06792,1011.04254,1011.09330,0.035893,NaN,NaN,NaN,NaN,0.983561,0.966553,0.989576,0.011342,NaN,NaN,NaN,NaN,74.178611,74.178611,74.178611,0.149748,0.000000,0.000000,0.000000,NaN,Bus,Pierpaolo
102,20104,445.0,NaN,5.0,9.075213,9.012243,9.187340,0.070605,0.907671,0.905519,0.909333,0.001954,9.806651,9.806651,9.806651,1.306210e-06,0.069503,0.025840,0.144650,0.048140,0.078626,0.019135,0.143299,0.040142,21.375,21.375,21.375,NaN,0.868719,0.658288,1.154273,0.207087,42.449803,41.281136,43.218342,0.606074,91.951478,91.951478,91.951478,0.651730,292.025891,291.819139,292.385244,0.312384,1010.27844,1010.27844,1010.27844,0.004688,NaN,NaN,NaN,NaN,0.787570,0.775354,0.802811,0.013976,31399.0,31399.0,31399.0,NaN,6.020600,6.020600,6.020600,0.000000,0.000000,0.000000,0.000000,0.0,Walking,Pierpaolo
121,19436,20.0,NaN,100.0,9.061146,8.246220,9.563967,0.451773,0.997597,0.997312,0.998010,0.000276,9.806651,9.806649,9.806653,1.650716e-06,0.045041,0.020783,0.099208,0.027367,0.038644,0.025420,0

In [25]:
# train data
df_train = df.merge(df_test, how='left', indicator=True)
df_train = df_train[df_train['_merge'] == 'left_only']
df_train=df_train.drop('_merge',axis=1)
df_train.head(5)

,id,time,activityrecognition_0,activityrecognition_1,android_sensor_accelerometer_mean,android_sensor_accelerometer_min,android_sensor_accelerometer_max,android_sensor_accelerometer_std,android_sensor_game_rotation_vector_mean,android_sensor_game_rotation_vector_min,android_sensor_game_rotation_vector_max,android_sensor_game_rotation_vector_std,android_sensor_gravity_mean,android_sensor_gravity_min,android_sensor_gravity_max,android_sensor_gravity_std,android_sensor_gyroscope_mean,android_sensor_gyroscope_min,android_sensor_gyroscope_max,android_sensor_gyroscope_std,android_sensor_gyroscope_uncalibrated_mean,android_sensor_gyroscope_uncalibrated_min,android_sensor_gyroscope_uncalibrated_max,android_sensor_gyroscope_uncalibrated_std,android_sensor_light_mean,android_sensor_light_min,android_sensor_light_max,android_sensor_light_std,android_sensor_linear_acceleration_mean,android_sensor_linear_acceleration_min,android_sensor_linear_acceleration_max,android_sensor_linear_acceleration_std,android_sensor_magnetic_field_mean,android_sensor_magnetic_field_min,android_sensor_magnetic_field_max,android_sensor_magnetic_field_std,android_sensor_magnetic_field_uncalibrated_mean,android_sensor_magnetic_field_uncalibrated_min,android_sensor_magnetic_field_uncalibrated_max,android_sensor_magnetic_field_uncalibrated_std,android_sensor_orientation_mean,android_sensor_orientation_min,android_sensor_orientation_max,android_sensor_orientation_std,android_sensor_pressure_mean,android_sensor_pressure_min,android_sensor_pressure_max,android_sensor_pressure_std,android_sensor_proximity_mean,android_sensor_proximity_min,android_sensor_proximity_max,android_sensor_proximity_std,android_sensor_rotation_vector_mean,android_sensor_rotation_vector_min,android_sensor_rotation_vector_max,android_sensor_rotation_vector_std,android_sensor_step_counter_mean,android_sensor_step_counter_min,android_sensor_step_counter_max,android_sensor_step_counter_std,sound_mean,sound_min,sound_max,sound_std,speed_mean,speed_min,speed_max,speed_std,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,0.029526,0.000119,9.806650,9.806649,9.806651,4.780692e-07,0.001651,0.000000,0.003533,0.000737,0.016221,0.014172,0.018695,0.000982,0.0,0.0,0.0,0.000000,0.020978,0.002495,0.052410,0.011045,57.099638,56.690387,57.575950,0.177549,51.363566,51.199707,51.539208,0.080899,354.286933,353.598335,354.942707,0.245676,1004.090261,1004.05540,1004.12790,0.017416,8.0,8.0,8.0,NaN,0.050413,0.044777,0.056351,0.002109,28966.0,28966.0,28966.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,Luca
1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,0.999946,0.000030,9.806624,9.806624,9.806624,6.474977e-07,0.036326,0.011669,0.059388,0.020290,0.039023,0.014132,0.085494,0.018629,0.0,0.0,0.0,NaN,0.879220,0.641117,1.185810,0.278730,29.351288,28.172505,30.386017,0.921547,82.767760,82.409890,83.125630,0.506105,332.695577,330.461054,339.108607,1.705816,1008.274660,1008.27466,1008.27466,NaN,NaN,NaN,NaN,NaN,0.999981,0.999963,0.999999,0.000026,NaN,NaN,NaN,NaN,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,Luca
2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,0.665218,0.000004,9.806650,9.806650,9.806650,2.821568e-07,0.001525,0.000000,0.002609,0.000713,0.016302,0.014645,0.017374,0.000876,2.0,2.0,2.0,0.707107,0.034110,0.025709,0.042511,0.011881,29.904927,29.741722,30.038309,0.130851,55.695716,55.501802,55.868506,0.128945,74.752653,73.223493,75.268574,0.413704,1005.977540,1005.97754,1005.97754,NaN,8.0,8.0,8.0,NaN,0.610456,0.610456,0.610456,0.011312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,Luca
3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,0.998661,0.003451,9.806638,9.806638,9.806638,2.120564e-06,0.036440,0.020463,0.052512,0.010553,0.050759,0.024732,0.081887,0.018037,0.0,0.0,0.0,NaN,1.488361,0.575910,2.415879,0.920077,

(4985, 70)
(908, 70)


In [27]:
unimp    = ['id','user','time']
df_train = df_train.drop(unimp,axis=1)
df_test  = df_test.drop(unimp,axis=1)

In [30]:
print(df_train.shape)
print(df_test.shape)

(4985, 67)
(908, 67)


In [28]:
X=df_train.drop('target',axis=1)
y=df_train.target
X_test=df_test.drop('target',axis=1)
y_test=df_test.target

In [31]:
X.shape

(4985, 66)

In [29]:
# label in coder

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [32]:

# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(15, input_dim=66, activation='relu'))
	model.add(Dense(5, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [35]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)

In [ ]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))